<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Parcial_2_SyS_2025-1/Punto2_Parcial2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎙️ Dashboard Interactivo de Modulación y Demodulación SSB-AM

**Punto 2 Parcial 2 SyS**  
**Omar Andrés Rodriguez Quiceno**  

---

En este notebook diseñamos y analizamos paso a paso el proceso de modulación y demodulación en amplitud de banda lateral única (SSB-AM), tanto en el dominio del tiempo como en el de la frecuencia. A través de una aplicación interactiva con **Streamlit**, el usuario podrá:

- Seleccionar la señal mensaje:
  1. Pulso rectangular  
  2. Segmento de 5 segundos de una canción  
- Visualizar cada etapa del sistema SSB-AM:
  - Señales en el tiempo (mensaje, portadora, modulada, filtrada y demodulada)  
  - Espectros correspondientes mediante transformada de Fourier  
- Ajustar parámetros clave:
  - Frecuencia de la portadora  
  - Ancho de banda del filtro (IIR) para selección de banda lateral  
- Inspeccionar características de los filtros digitales recursivos:
  - Diagrama de Bode  
  - Plano de polos y ceros  
- Comprender la equivalencia entre teoría y simulación, gracias a gráficos claros y descripciones concretas de cada bloque funcional.



## Instalación de Dependencias y Configuración Inicial

En este bloque instalamos las librerías necesarias para el procesamiento de señales y la construcción del dashboard, y preparamos la herramienta **Cloudflared** para exponer la aplicación:

1. Instalamos **Streamlit** para el dashboard interactivo y **Librosa** para la carga y análisis de audio.  
2. Descargamos el binario oficial de **Cloudflared**, lo hacemos ejecutable y lo movemos a `/usr/local/bin` para poder crear un túnel público más adelante.


In [25]:
# ── Instalación de librerías para la aplicación ──────────────────────────
!pip install streamlit librosa --quiet
                                     # Streamlit: interfaz web
                                     # Librosa: carga y análisis de audio

# ── Descarga y configuración de Cloudflared ─────────────────────────────
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
                                     # Descarga la última versión del binario
!chmod +x cloudflared                # Asigna permiso de ejecución al archivo
!mv cloudflared /usr/local/bin/cloudflared
                                     # Mueve el binario a una carpeta del PATH


## Script Principal: `1_📊_SSB_AM_Dashboard.py`

En este archivo definimos la aplicación Streamlit para visualizar el proceso de modulación y demodulación SSB-AM. El usuario podrá:

1. **Seleccionar la señal mensaje**  
   - Pulso rectangular de duración configurable  
   - Fragmento de 5 s de un archivo .wav  
2. **Ajustar parámetros clave** en la barra lateral:  
   - Frecuencia de la portadora \(f_c\)  
   - Frecuencia de corte del filtro pasabajo IIR  
3. **Explorar cada etapa del sistema** mediante secciones desplegables:  
   - Forma de la señal mensaje en el tiempo  
   - Señal SSB-AM modulada  
   - Señal demodulada y filtrada  
   - Análisis del filtro digital (diagrama de Bode y plano z de polos y ceros)  

Cada sección incluye gráficos y una breve descripción que explica el propósito y la interpretación de los resultados, asegurando una comprensión clara del proceso SSB-AM.


In [28]:
%%writefile 1_📊_SSB_AM_Dashboard.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.signal import hilbert, butter, lfilter, freqz

# ── Auxiliares ────────────────────────────────────────────
def rectangular_pulse(duration=1.0, fs=1000):
    t = np.linspace(0, duration, int(fs*duration), endpoint=False)
    return t, np.ones_like(t)

def load_audio_segment(path, duration=5):
    y, sr = librosa.load(path, duration=duration)
    return y, sr

def ssb_modulate(m, fs, fc):
    t = np.arange(len(m)) / fs
    mh = np.imag(hilbert(m))
    return t, m * np.cos(2*np.pi*fc*t) - mh * np.sin(2*np.pi*fc*t)

def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    wn  = cutoff / nyq
    return butter(order, wn, btype='low', analog=False)

def apply_filter(x, b, a):
    return lfilter(b, a, x)

# ── Página ─────────────────────────────────────────────────
st.set_page_config("SSB-AM Dashboard", layout="wide")
st.sidebar.header("⚙️ Parámetros")

# Parámetros de usuario
modo   = st.sidebar.radio("Señal mensaje", ["Pulso rectangular", "Fragmento .wav"])
fs     = 1000
fc     = st.sidebar.slider("Portadora fc [Hz]", 50, 5000, 1000)
cutoff = st.sidebar.slider("Corte LP [Hz]", 10, fs//2, 200)

# ── Carga de señal ─────────────────────────────────────────
if modo == "Pulso rectangular":
    t, msg = rectangular_pulse(fs=fs)
else:
    wav = st.sidebar.file_uploader("Sube un .wav", type=["wav"])
    if not wav:
        st.sidebar.warning("Carga un archivo para continuar")
        st.stop()
    msg, fs = load_audio_segment(wav)
    t = np.arange(len(msg)) / fs
    st.audio(wav)

# ── Encabezado ─────────────────────────────────────────────
st.title("📻 Visualización SSB-AM")
st.markdown("Selecciona los parámetros en la barra lateral y explora cada etapa:")

# ── 1) Señal Mensaje ───────────────────────────────────────
with st.expander("📈 1. Señal Mensaje", expanded=True):
    st.line_chart(msg)
    st.markdown("""
    **📝 Descripción:** Forma de onda de la señal original.
    **🔎 Interpretación:** Es la información base que queremos transmitir.
    """)

# ── 2) Señal SSB-Modulada ───────────────────────────────────
t_ssb, ssb = ssb_modulate(msg, fs, fc)
with st.expander("📡 2. Señal Modulada (SSB-AM)"):
    fig, ax = plt.subplots(figsize=(8,2))
    ax.plot(t_ssb, ssb)
    ax.set(xlabel="t [s]", ylabel="s(t)")
    ax.grid()
    st.pyplot(fig)
    st.markdown("""
    **📝 Descripción:** Señal modulada en banda lateral única.
    **🔎 Interpretación:** Se elimina una banda lateral para reducir el ancho de banda.
    """)

# ── 3) Demodulación + Filtrado ─────────────────────────────
dem   = ssb * 2 * np.cos(2*np.pi*fc*t_ssb)
b, a  = butter_lowpass(cutoff, fs)
demf  = apply_filter(dem, b, a)
with st.expander("🔧 3. Señal Demodulada y Filtrada"):
    fig, ax = plt.subplots(figsize=(8,2))
    ax.plot(t_ssb, demf)
    ax.set(xlabel="t [s]", ylabel="m̂(t)")
    ax.grid()
    st.pyplot(fig)
    st.markdown("""
    **📝 Descripción:** Envolvente recuperada tras demodulación y filtro IIR pasabajo.
    **🔎 Interpretación:** El filtro elimina componentes de alta frecuencia no deseadas.
    """)

# ── 4) Análisis del filtro digital ─────────────────────────
with st.expander("📉 4. Análisis del Filtro Digital"):
    col1, col2 = st.columns(2)
    with col1:
        w, h = freqz(b, a, fs=fs)
        fig, ax = plt.subplots(figsize=(4,3))
        ax.semilogx(w, 20*np.log10(np.abs(h)))
        ax.set(title="Bode Magnitud", xlabel="Hz", ylabel="dB")
        ax.grid()
        st.pyplot(fig)
        st.markdown("""
        **📝 Descripción:** Diagrama de Bode del filtro digital IIR.
        **🔎 Interpretación:** Muestra atenuación o ganancia según la frecuencia.
        """)
    with col2:
        zeros_d = np.roots(b)
        poles_d = np.roots(a)
        fig, ax = plt.subplots(figsize=(4,4))
        theta = np.linspace(0,2*np.pi,200)
        ax.plot(np.cos(theta), np.sin(theta), 'k--')
        ax.scatter(np.real(zeros_d), np.imag(zeros_d), facecolors='none', edgecolors='b', label='Ceros')
        ax.scatter(np.real(poles_d), np.imag(poles_d), marker='x', color='r', label='Polos')
        ax.set(title="Plano Z", xlabel="Re(z)", ylabel="Im(z)")
        ax.axhline(0); ax.axvline(0)
        ax.legend(); ax.grid()
        st.pyplot(fig)
        st.markdown("""
        **📝 Descripción:** Ubicación de polos (✖) y ceros (○) en el plano z.
        **🔎 Interpretación:** Polos dentro del círculo unitario garantizan estabilidad.
        """)


Overwriting 1_📊_SSB_AM_Dashboard.py


## Ejecución de la Aplicación y Exposición Pública

En este bloque ponemos en marcha el dashboard de SSB-AM y creamos un túnel público con Cloudflared para acceder desde cualquier dispositivo:

1. **Cierre de procesos previos** de Streamlit y Cloudflared para evitar conflictos.  
2. **Ejecución** del script principal en segundo plano, omitiendo los logs en pantalla.  
3. **Pausa** breve para asegurar que el servidor local de Streamlit esté activo.  
4. **Arranque** del túnel con Cloudflared apuntando al puerto 8501.  
5. **Lectura** del archivo de log para extraer y mostrar la URL pública generada.  


In [29]:
# 🟢 Ejecuta la app y crea el túnel público

import os, time, re

# 1) Cerrar instancias antiguas de Streamlit y Cloudflared
!pkill -f streamlit               # Finaliza cualquier servidor Streamlit en ejecución
!pkill -f cloudflared             # Finaliza túneles previos de Cloudflared

# 2) Iniciar la app Streamlit en background (sin mostrar logs)
!streamlit run 1_📊_SSB_AM_Dashboard.py &> /dev/null &

# 3) Esperar unos segundos para que Streamlit arranque correctamente
time.sleep(3)

# 4) Lanzar el túnel con el binario oficial de Cloudflared
!cloudflared tunnel --url http://localhost:8501 > cloudflared.log 2>&1 &

# 5) Dar tiempo para que Cloudflared genere la URL pública
time.sleep(8)

# 6) Leer el log y extraer la URL
url = None
with open("cloudflared.log") as f:
    for line in f:
        m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
        if m:
            url = m.group(0)
            break

# 7) Mostrar el resultado al usuario
if url:
    print("✅ Tu dashboard está disponible aquí:\n", url)
else:
    print("⚠️ No se pudo obtener la URL. Verifica 'cloudflared.log' para más detalles.")


✅ Tu dashboard está disponible aquí:
 https://backed-relief-operated-script.trycloudflare.com


## Conclusiones

- A través de este ejercicio profundizamos en el modelo matemático de la modulación SSB-AM, observando cómo la combinación de la señal original y su transformada de Hilbert genera la eliminación de una banda lateral tanto en el dominio del tiempo como en el de la frecuencia.

- El dashboard permite explorar dinámicamente el efecto de cambiar la frecuencia de la portadora y la frecuencia de corte del filtro pasabajo IIR. Esto facilita la intuición sobre cómo esos parámetros afectan la forma de onda modulada y la calidad de la señal demodulada.

- Implementamos un filtro Butterworth de orden configurable, y comprobamos su respuesta mediante diagramas de Bode y el plano Z de polos y ceros. De esta manera, reforzamos la comprensión de criterios de estabilidad y selectividad en filtros IIR.
